<a href="https://colab.research.google.com/github/michael-vdr/Twittorials/blob/master/Quick_win_wordcount_meta_title_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





### Resources
You can find example CSV files in the sheet: https://docs.google.com/spreadsheets/d/18e5Xd1l2dHojd91L7e1saY0xGufsIdhPUYe8yDL88Ag/edit?usp=sharing


### Step 0: Language settings

In [ ]:
langcode="en" #@param {type:"string"}
language="english" #@param {type:"string"}

### Step 1 : Import modules

In [ ]:
#import modules
import pandas as pd 
#input/output modules for uploading and downloading files
from google.colab import files
import io
#install nlp modules not yet available in colab
!pip install stop_words
!pip install python-levenshtein
import re #NLP regex module
import nltk #general NLP module
from nltk.stem.snowball import SnowballStemmer #NLP stemming module
from Levenshtein import * #NLP distance caluclator
from stop_words import get_stop_words #stop words NLP library
nltk.download('punkt') #tokenizer NLP 
nltk.download('stopwords')

### Step 2 : Upload the page title file

In [ ]:
#Step 2: upload the page title file
uploaded = files.upload()
filename = str(uploaded).split("'")[1]
dftitles = pd.read_csv(io.BytesIO(uploaded[filename]))
dftitles.columns=["Page","Title"]
dftitles.head()

In [ ]:
str(uploaded)

### Step 3: Upload second CSV file: the search queries by page 

In [ ]:
uploaded = files.upload()
filename = str(uploaded).split("'")[1]
dfqueries = pd.read_csv(io.BytesIO(uploaded[filename]))
dfqueries.columns=["Query","Page"]
dfqueries.head()

### Step 4: Combining dataframes

In [ ]:
df = pd.merge(dfqueries,dftitles,how="inner",on='Page')
df.head()

### Step 5: Count number of words

In [ ]:
stop_words = get_stop_words(langcode)

def number_words(sentence):
  sentencewords = nltk.word_tokenize(str(sentence).lower()) 
  not_stop_words=[]
  for word in sentencewords:
    if(word not in stop_words):
        not_stop_words.append(word)
  sentencelength = len(not_stop_words)
  return sentencelength

df["Words in query"]=df.apply(lambda x:number_words(x["Query"]),axis=1)
df["Words in title"]=df.apply(lambda x:number_words(x["Title"]),axis=1)
df.head()

### Step 6: Count number of common words
words of query in title

In [ ]:
def common_words(query,title):
    stemmer  = SnowballStemmer(language, ignore_stopwords=True)
    titlewords = nltk.word_tokenize(str(title).lower())
    stemtitlewords = [stemmer.stem(word) for word in titlewords]
    querywords = nltk.word_tokenize(str(query).lower()) 
    stemquerywords = [stemmer.stem(word) for word in querywords]
    titlenospaces=title.replace(' ','').lower()
    title_only_text = re.sub(r'\W+', '', titlenospaces)
    
    lev_distance=0  
    commonwords=0

    for word in stemquerywords:
      if(word not in stop_words):
          #literal word check
          if(word in title_only_text):
              commonwords=commonwords+1         
          else:
          #check with word with title stemmed and for misspellings
            for word2 in stemtitlewords:
                lev_distance = distance(str(word),str(word2))
                procent_distance = lev_distance/(len(word)+len(word2)/2)
                if procent_distance<0.15:
                    commonwords=commonwords+1
    return commonwords
    
df["Common words"]=df.apply(lambda x:common_words(x["Query"],x["Title"]),axis=1)
df.sample(5)

### Step 7 sort and explore table

In [ ]:
%load_ext google.colab.data_table

print("Possible optimizations")
print("----------------------")

df=df.sort_values(by=['Common words', 'Words in query'], ascending=[True, False])
df

### Additional step: Export and download

In [ ]:
#automatically download
df.to_csv('export.csv')
files.download('export.csv') 